In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Regarding messed up leader board, a mistake I made and how to make your validation score and learder board score correctly reflect your model 
This kernel is written due to a wrong answer I gave to a comment under another kaggler's kernel. So to make it right and cosidering that I was also lost in the same wrong path for a while, I think it's necessary to show where is the problem, what it can cause and how to properly handle it.  

To be brief, there are leakage problems in some kernels here. And I think this is part of the reason there are some god like score in leader board right now. This causes some obstacle in evaluate the performance of model. Below is the layout of this kernel:  
* [Data Prep](#dp)
* [Issue](#i)
* [Comparison Code](#cc)
* [Conclusion](http://)

# <a name ="dp"></a>Data Prep

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
import lightgbm as lgb
import string

In [ ]:
path = '../input/google-cloud-ncaa-march-madness-2020-division-1-womens-tournament/'
regular_result_brief=pd.read_csv(path+'WDataFiles_Stage1/WRegularSeasonCompactResults.csv')
tour_result_brief=pd.read_csv(path+'WDataFiles_Stage1/WNCAATourneyCompactResults.csv')
tour_seeds=pd.read_csv(path+'WDataFiles_Stage1/WNCAATourneySeeds.csv')
seasons=pd.read_csv(path+'WDataFiles_Stage1/WSeasons.csv')
sub_test=pd.read_csv(path+'WSampleSubmissionStage1_2020.csv')

In [ ]:
regular_result_brief.drop(['WLoc','NumOT'],axis=1,inplace=True)
tour_result_brief.drop(['WLoc','NumOT'],axis=1,inplace=True)
rwin=regular_result_brief[['Season', 'DayNum', 'WTeamID', 'WScore']]
rwin.columns=['Season', 'DayNum', 'TeamID', 'Score']
rlos=regular_result_brief[['Season', 'DayNum', 'LTeamID', 'LScore']]
rlos.columns=['Season', 'DayNum', 'TeamID', 'Score']
regular_score=pd.concat((rwin,rlos)).reset_index(drop=True)
regular_score=regular_score.groupby(['Season','TeamID'],as_index=False).mean()
regular_score.head(7)

In [ ]:
tour_seeds['Seed']=tour_seeds['Seed'].apply(lambda x: int(x[1:]))
def myMerge(indf):
    df=indf[['Season', 'WTeamID', 'LTeamID']]
    df=df.merge(tour_seeds,left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
    df.rename(columns={'Seed':'WSeed'}, inplace=True)
    df = df.drop('TeamID', axis=1)
    df = df.merge(tour_seeds, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
    df.rename(columns={'Seed':'LSeed'}, inplace=True)
    df = df.drop('TeamID', axis=1)
    df = pd.merge(df, regular_score, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left')
    df.rename(columns={'Score':'WScore','DayNum':'WDayNum'}, inplace=True)
    df = df.drop(['TeamID'], axis=1)
    df = pd.merge(df, regular_score, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left')
    df.rename(columns={'Score':'LScore','DayNum':'LDayNum'}, inplace=True)
    df = df.drop(['TeamID'], axis=1)
    return df
tour_merge=myMerge(tour_result_brief)
tour_merge.head(7)

In [ ]:
tour_win=tour_merge.copy()
tour_los=tour_merge.copy()
tour_win.columns=[c.replace('W','T1_').replace('L','T2_') for c in tour_win.columns]
tour_los.columns=[c.replace('W','T2_').replace('L','T1_') for c in tour_los.columns]
tour_win['Pred']=1
tour_los['Pred']=0
tour_all=pd.concat((tour_win, tour_los),sort=True).reset_index(drop=True)
tour_all

In [ ]:
sub_test['Season'] = sub_test['ID'].map(lambda x: int(x[:4]))
sub_test['WTeamID'] = sub_test['ID'].map(lambda x: int(x[5:9]))
sub_test['LTeamID'] = sub_test['ID'].map(lambda x: int(x[10:14]))
test=myMerge(sub_test)
test.columns=[c.replace('W','T1_').replace('L','T2_') for c in test.columns]
test.head(7)

In [ ]:
feature_dungeon=['ID', 'Pred', 'T1_TeamID', 'T2_TeamID']
features=[c for c in tour_all.columns if c not in feature_dungeon]
target=tour_all.Pred
train=tour_all[features]
test=test[features]

In [ ]:
print('train shape: {}, test shape: {}'.format(train.shape,test.shape))

# <a name ="i"></a>Issue
Up until now every thing is fine. Then some kernel did a n fold cross validation on this entire taining set and get the prediction from test set. Using this procedure and the data above, my submission can get about 0.268 on leader board. But if you look back the competition in 2018 and 2019, the leader score 0.4 and 0.32 respectively.  
The reason that I have a low log loss here is that some of the test data have duplication in training set. Due to the same reason, another thing you can observe is that your leader board score have a strong link with the training loss instead validation loss. Also The other thing you can see is training log loss and leader board score excitedly react to the increase of number of folds, whereas validation loss doesn't move much.

# <a name ="cc"></a>Comparision Code
Below is the similar code in some kenel. Here I use folds=100 to save time(I have seen even larger number) 

In [ ]:
params = {
        'learning_rate': 0.01,
        'feature_fraction': 0.5,
        'min_data_in_leaf' : 12,
        'max_depth': 8,
        'objective': 'binary',
        'metric': 'binary_logloss',
        'n_jobs': -1,
        'feature_fraction_seed': 42,
        'bagging_seed': 42,
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'is_unbalance': False,
        'boost_from_average': False}
traintion = np.zeros(len(train))
validation = np.zeros(len(train))
predictions = np.zeros(len(test))
feature_importance_df = pd.DataFrame()
folds = KFold(n_splits=100, shuffle=True, random_state=1990)
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train)):
    print("fold n°{}".format(fold_))
    train_x=train.iloc[trn_idx].reset_index(drop=True)
    valid_x=train.iloc[val_idx].reset_index(drop=True)
    target_train=target.iloc[trn_idx].reset_index(drop=True)
    target_valid=target.iloc[val_idx].reset_index(drop=True)
    trn_data = lgb.Dataset(train_x,
                           label=target_train,
                          )
    val_data = lgb.Dataset(valid_x,
                           label=target_valid,
                          )

    num_round = 1000000
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=False,
                    early_stopping_rounds = 1000)
    traintion[trn_idx] += clf.predict(train_x, num_iteration=clf.best_iteration)/(folds.n_splits-1)
    validation[val_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test, num_iteration=clf.best_iteration) / folds.n_splits
print("Train AUC score: {:<8.5f}".format(log_loss(target,traintion)))
print("Valid AUC score: {:<8.5f}".format(log_loss(target,validation)))

# <a name ="c"></a>Conclusion
The solution is simple. Just using data from previous seasons to predict the season of interest. For example, using data from 2010-2014 seasons in training set build model and predict season 2015 and put it in submission file, using 2010-2015 to do 2016 so on so forth.
Then you will find your validation score makes much sense with respect to your leader board score. 
And if you want to compare your model to others and have an idea of how it is, you can reference previous competitions, since current public leader board is a bit messed up.  
[2018 leader board](https://www.kaggle.com/c/womens-machine-learning-competition-2018/leaderboard)  
[2019 leader board](https://www.kaggle.com/c/womens-machine-learning-competition-2019/leaderboard)

#### Hope this post helped you! 